In [1]:
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
virus_DF = pd.read_csv('virus_data.csv')
virus_DF.shape

(1250, 25)

In [3]:
virus_DF["conversations_per_day"].value_counts()

conversations_per_day
3     218
2     204
5     179
4     168
1     108
6     107
7      94
8      54
9      42
10     29
11     16
13      8
12      7
14      6
16      5
15      3
17      1
29      1
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
sum = 0
for id in virus_DF["patient_id"]:
    sum+= id%100

train_virus_df, test_virus_df = train_test_split(virus_DF, test_size=0.2, random_state=sum)
